<a href="https://colab.research.google.com/github/manasc12/RoadMishapRiskAssessment/blob/master/RandomForest_Implementation_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Implementing Random Forest Classifier </h1>
<h3> Importing Required Libraries </h3>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import datasets

<h3> Reading the Data from CSV file </h3>

In [ ]:
df_Acc=pd.read_csv('/Users/manasc12/MyProjects/Data/Accidents0515.csv')
df_Acc.head(5)

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
0,200501BS00001,525680.0,178240.0,-0.191170,51.489096,1,2,1,1,04/01/2005,...,0,1,1,2,2,0,0,1,1,E01002849
1,200501BS00002,524170.0,181650.0,-0.211708,51.520075,1,3,1,1,05/01/2005,...,0,5,4,1,1,0,0,1,1,E01002909
2,200501BS00003,524520.0,182240.0,-0.206458,51.525301,1,3,2,1,06/01/2005,...,0,0,4,1,1,0,0,1,1,E01002857
3,200501BS00004,526900.0,177530.0,-0.173862,51.482442,1,3,1,1,07/01/2005,...,0,0,1,1,1,0,0,1,1,E01002840
4,200501BS00005,528060.0,179040.0,-0.156618,51.495752,1,3,1,1,10/01/2005,...,0,0,7,1,2,0,0,1,1,E01002863


<h3> Data Cleaning and Wrangling

In [ ]:
df_Acc_clnd=df_Acc[~df_Acc.isnull().any(axis=1)]

In [ ]:
a= np.array([(int(str(x).split(':')[0])*60+int(str(x).split(':')[1])) for x in df_Acc_clnd['Time'].values])

In [ ]:
df_Acc_clnd['Time_Conv']=a

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_Acc_clnd.nunique(axis=0)

Accident_Index                                 1651142
Location_Easting_OSGR                           203321
Location_Northing_OSGR                          229918
Longitude                                      1143738
Latitude                                       1064222
Police_Force                                        48
Accident_Severity                                    3
Number_of_Vehicles                                  28
Number_of_Casualties                                51
Date                                              4017
Day_of_Week                                          7
Time                                              1439
Local_Authority_(District)                         391
Local_Authority_(Highway)                          182
1st_Road_Class                                       6
1st_Road_Number                                   6951
Road_Type                                            6
Speed_limit                                          9
Junction_D

In [ ]:
df_Acc_clnd.dtypes

Accident_Index                                  object
Location_Easting_OSGR                          float64
Location_Northing_OSGR                         float64
Longitude                                      float64
Latitude                                       float64
Police_Force                                     int64
Accident_Severity                                int64
Number_of_Vehicles                               int64
Number_of_Casualties                             int64
Date                                            object
Day_of_Week                                      int64
Time                                            object
Local_Authority_(District)                       int64
Local_Authority_(Highway)                       object
1st_Road_Class                                   int64
1st_Road_Number                                  int64
Road_Type                                        int64
Speed_limit                                      int64
Junction_D

In [ ]:
df_Acc_clnd.drop(['Date','Time','Accident_Index','Location_Easting_OSGR','Location_Northing_OSGR'],axis=1,inplace=True)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df_Acc_clnd.shape

(1651142, 28)

In [ ]:
df_Acc_clnd.dtypes

Longitude                                      float64
Latitude                                       float64
Police_Force                                     int64
Accident_Severity                                int64
Number_of_Vehicles                               int64
Number_of_Casualties                             int64
Day_of_Week                                      int64
Local_Authority_(District)                       int64
Local_Authority_(Highway)                       object
1st_Road_Class                                   int64
1st_Road_Number                                  int64
Road_Type                                        int64
Speed_limit                                      int64
Junction_Detail                                  int64
Junction_Control                                 int64
2nd_Road_Class                                   int64
2nd_Road_Number                                  int64
Pedestrian_Crossing-Human_Control                int64
Pedestrian

<h3> Bifurcating Attributes for Training and Target

In [ ]:
df=df_Acc_clnd.iloc[0:100000,:].reset_index(drop=True)
target_c=['Accident_Severity']
df_train=df.drop(target_c,axis=1)
df_target=df[target_c]

<h3> Preparing Training Data with LabelEncoding and OneHotEncoding

In [ ]:
pd.get_dummies(df_train)#to be honest this didnt solve my purpose so I had to do One-Hot-Encoding

,Longitude,Latitude,Police_Force,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Local_Authority_(District),1st_Road_Class,1st_Road_Number,Road_Type,...,LSOA_of_Accident_Location_E01032476,LSOA_of_Accident_Location_E01032477,LSOA_of_Accident_Location_E01032478,LSOA_of_Accident_Location_E01032479,LSOA_of_Accident_Location_E01032480,LSOA_of_Accident_Location_E01032481,LSOA_of_Accident_Location_E01032482,LSOA_of_Accident_Location_W01000121,LSOA_of_Accident_Location_W01000328,LSOA_of_Accident_Location_W01000332
0,-0.191170,51.489096,1,1,1,3,12,3,3218,6,...,0,0,0,0,0,0,0,0,0,0
1,-0.211708,51.520075,1,1,1,4,12,4,450,3,...,0,0,0,0,0,0,0,0,0,0
2,-0.206458,51.525301,1,2,1,5,12,5,0,6,...,0,0,0,0,0,0,0,0,0,0
3,-0.173862,51.482442,1,1,1,6,12,3,3220,6,...,0,0,0,0,0,0,0,0,0,0
4,-0.156618,51.495752,1,1,1,2,12,6,0,6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-1.016265,53.357269,31,2,1,7,341,3,634,6,...,0,0,0,0,0,0,0,0,0,0
99996,-0.998357,53.204482,31,2,1,6,345,3,6075,6,...,0,0,0,0,0,0,0,0,0,0
99997,-1.118040,53.099622,31,2,3,6,345,4,6020,6,...,0,0,0,0,0,0,0,0,0,0
99998,-0.924000,53.226217,31,3,1,5,341,3,6075,6,...,0,0,0,0,0,0,0,0,0,0


<h4> 1. LabelEncoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelEncodeCols=['LSOA_of_Accident_Location','Local_Authority_(Highway)']
Local_Authority_Highway               = LabelEncoder()
LSOA_of_Accident_Location               = LabelEncoder()

Local_Authority_Highway.fit(df_train['Local_Authority_(Highway)'])
LSOA_of_Accident_Location.fit(df_train['LSOA_of_Accident_Location'])

df_train['Local_Authority_(Highway)_E']=Local_Authority_Highway.transform(df_train['Local_Authority_(Highway)'])
df_train['LSOA_of_Accident_Location_E']=LSOA_of_Accident_Location.transform(df_train['LSOA_of_Accident_Location'])
df_train.drop(labelEncodeCols,axis=1,inplace=True)


In [ ]:
df_train.nunique(axis=0)

Longitude                                      89712
Latitude                                       88497
Police_Force                                      20
Number_of_Vehicles                                13
Number_of_Casualties                              19
Day_of_Week                                        7
Local_Authority_(District)                       171
1st_Road_Class                                     6
1st_Road_Number                                 2331
Road_Type                                          6
Speed_limit                                        7
Junction_Detail                                   10
Junction_Control                                   6
2nd_Road_Class                                     7
2nd_Road_Number                                 2169
Pedestrian_Crossing-Human_Control                  4
Pedestrian_Crossing-Physical_Facilities            7
Light_Conditions                                   5
Weather_Conditions                            

<h4> 2. OneHotEncoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
#from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
#ct = ColumnTransformer([(('Police_Force', 'Number_of_Vehicles'), OneHotEncoder(),[2,3])], remainder = 'passthrough')
noOneHotEncCols=['Longitude','Latitude','Local_Authority_(District)','1st_Road_Number','2nd_Road_Number','Time_Conv','Local_Authority_(Highway)_E','LSOA_of_Accident_Location_E']
mct = make_column_transformer(( OneHotEncoder(),df_train.drop(noOneHotEncCols,axis=1).columns.to_numpy()), remainder = 'passthrough')
train=mct.fit_transform(df_train).toarray()
target=df_target.values.reshape(len(df_target))


<h3> Splitting up Training and Testing Data Samples

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train,target,test_size=0.1,random_state=42) #OneHotEncoded Data Samples
x1_train, x1_test, y1_train, y1_test = train_test_split(df_train,df_target,test_size=0.1,random_state=42) #Orignal LabelEncoded Data Samples

<h3> Implementing Random Forest Classifier
    <h4> 1. Model with OneHotEncoded Data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_W_Ohe = RandomForestClassifier(n_estimators=50)
model_W_Ohe.fit(x_train, y_train)

RandomForestClassifier(n_estimators=50)

In [ ]:
model_W_Ohe.score(x_test, y_test)

0.8675

In [ ]:
from sklearn import metrics

metrics.confusion_matrix(y_test, model_W_Ohe.predict(x_test), labels=[3, 2, 1])

array([[8655,   55,    3],
       [1159,   20,    0],
       [ 104,    4,    0]])

<h4> Geting the Feature Importance

In [ ]:
model_W_Ohe.feature_importances_

array([2.51715893e-03, 1.47693320e-03, 3.06667380e-03, 2.39227948e-03,
       2.88862568e-03, 2.98284887e-03, 2.13952092e-03, 1.70772667e-03,
       2.69471474e-03, 3.66406247e-03, 2.82723711e-03, 2.43115194e-03,
       1.16259289e-03, 2.79552982e-03, 2.17465968e-03, 2.28034270e-03,
       1.42087147e-03, 2.47937540e-03, 9.78094720e-04, 3.20730557e-04,
       1.38125161e-02, 1.03118657e-02, 4.81113791e-03, 1.85199031e-03,
       7.55060361e-04, 4.46516653e-04, 1.14174315e-04, 1.73105969e-04,
       1.29080254e-06, 2.34172585e-06, 1.05468378e-05, 1.63630576e-08,
       1.40962089e-05, 1.07285895e-02, 8.68433542e-03, 5.43339681e-03,
       4.10217013e-03, 2.49274515e-03, 1.45213267e-03, 7.35371160e-04,
       3.85756382e-04, 1.75859108e-04, 3.62291463e-06, 5.78492984e-05,
       3.12608879e-05, 1.90864514e-06, 9.82650651e-07, 5.67013642e-05,
       4.27230486e-05, 5.53496556e-05, 0.00000000e+00, 1.58541248e-06,
       1.07422937e-02, 1.13343155e-02, 1.16394343e-02, 1.16671834e-02,
      

<h4>2. Model with LabelEncoded Data

In [ ]:
model_W_Le = RandomForestClassifier(n_estimators=50)
model_W_Le.fit(x1_train, y_train)

RandomForestClassifier(n_estimators=50)

In [ ]:
model_W_Le.score(x1_test, y1_test)

0.868

In [ ]:
metrics.confusion_matrix(y1_test, model_W_Le.predict(x1_test), labels=[3, 2, 1])

array([[8645,   67,    1],
       [1143,   35,    1],
       [  99,    9,    0]])

<h4> Getting The Feature Importance
<h5> The Code below is only Usable if you have Data Frames( i.e. x1_train , y1_train ) as Traning and ( i.e. x1_test , y1_test ) as Testing Data.

In [ ]:
# Get numerical feature importances
importances = list(model_W_Le.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(x1_train.columns, importances)]

In [ ]:
feature_importances

[('Longitude', 0.12),
 ('Latitude', 0.12),
 ('Police_Force', 0.02),
 ('Number_of_Vehicles', 0.03),
 ('Number_of_Casualties', 0.03),
 ('Day_of_Week', 0.06),
 ('Local_Authority_(District)', 0.05),
 ('1st_Road_Class', 0.02),
 ('1st_Road_Number', 0.07),
 ('Road_Type', 0.02),
 ('Speed_limit', 0.02),
 ('Junction_Detail', 0.02),
 ('Junction_Control', 0.01),
 ('2nd_Road_Class', 0.01),
 ('2nd_Road_Number', 0.02),
 ('Pedestrian_Crossing-Human_Control', 0.0),
 ('Pedestrian_Crossing-Physical_Facilities', 0.02),
 ('Light_Conditions', 0.02),
 ('Weather_Conditions', 0.02),
 ('Road_Surface_Conditions', 0.02),
 ('Special_Conditions_at_Site', 0.01),
 ('Carriageway_Hazards', 0.0),
 ('Urban_or_Rural_Area', 0.01),
 ('Did_Police_Officer_Attend_Scene_of_Accident', 0.01),
 ('Time_Conv', 0.12),
 ('Local_Authority_(Highway)_E', 0.04),
 ('LSOA_of_Accident_Location_E', 0.11)]

In [ ]:
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)


In [ ]:

# Print out the feature and importances 
[print('Variable--------{}: Importance: {}'.format(*pair)) for pair in feature_importances];

Variable--------Longitude: Importance: 0.12
Variable--------Latitude: Importance: 0.12
Variable--------Time_Conv: Importance: 0.12
Variable--------LSOA_of_Accident_Location_E: Importance: 0.11
Variable--------1st_Road_Number: Importance: 0.07
Variable--------Day_of_Week: Importance: 0.06
Variable--------Local_Authority_(District): Importance: 0.05
Variable--------Local_Authority_(Highway)_E: Importance: 0.04
Variable--------Number_of_Vehicles: Importance: 0.03
Variable--------Number_of_Casualties: Importance: 0.03
Variable--------Police_Force: Importance: 0.02
Variable--------1st_Road_Class: Importance: 0.02
Variable--------Road_Type: Importance: 0.02
Variable--------Speed_limit: Importance: 0.02
Variable--------Junction_Detail: Importance: 0.02
Variable--------2nd_Road_Number: Importance: 0.02
Variable--------Pedestrian_Crossing-Physical_Facilities: Importance: 0.02
Variable--------Light_Conditions: Importance: 0.02
Variable--------Weather_Conditions: Importance: 0.02
Variable--------

<h3> Visualize a random Tree from the Forest

In [ ]:

# Import tools needed for visualization
#from sklearn.tree import export_graphviz
#import pydot

# Pull out one tree from the forest
#tree = model_W_Le.estimators_[5]

# Export the image to a dot file
#export_graphviz(tree, out_file = 'tree.dot', feature_names = x1_train.columns, rounded = True, precision = 1)





In [ ]:
# Use dot file to create a graph
#(graph, ) = pydot.graph_from_dot_file('tree.dot')

In [ ]:
# Write graph to a png file
#graph.write_png('tree.jpg')

<h3> Now Lets Implement Our Classifier with some HyperParameters using Parameter Tuning</h3>
<h7>-----------------------------------------------------------------------------------------------------------------------------------------------------------------</h7>
<h7><br> We tried to implement Random Forest Classifier on both OneHotEncoded and Label-Encoded Data. But the results are not looking promising for: <br>Accident Severity=2 <b>(Serious)</b> and Accident Severity=1 <b>(Fatal)</b></h7>

<h4>3. Model with LabelEncoded Data + HyperParameters + RandomizedSearchCV + CrossValidation 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [20, 28, 37, 46, 55, 64, 73, 82, 91, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Use the random grid to search for best hyperparameters
# First create the base model to tune
model_W_Le_Hp = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
model_W_Le_Hp_Rnd = RandomizedSearchCV(estimator = model_W_Le_Hp, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
model_W_Le_Hp_Rnd.fit(x1_train, y1_train)


In [ ]:
from joblib import dump, load #to save ur model
dump(model_W_Le_Hp_Rnd, '/content/models/RF_model_W_Le_Hp_Rnd.joblib') 
#model_W_Le_Hp_Rnd=load('/content/models/RF_model_W_Le_Hp_Rnd.joblib')


In [ ]:
#View the best parameters of the best estimator
model_W_Le_Hp_Rnd.best_params_

{'n_estimators': 55,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [ ]:
#Test ur model with the best estimator 
metrics.accuracy_score(y1_test, model_W_Le_Hp_Rnd.best_estimator_.predict(x1_test))

0.8713

In [ ]:
#Getting the predicted value counts
pd.value_counts(pd.DataFrame({'A':model_W_Le_Hp_Rnd.best_estimator_.predict(x1_test)})['A'],sort=True)

3    10000
Name: A, dtype: int64

In [ ]:
#Getting the confusion matrix
metrics.confusion_matrix(y1_test, model_W_Le_Hp_Rnd.best_estimator_.predict(x1_test), labels=[3, 2, 1])

array([[8713,    0,    0],
       [1179,    0,    0],
       [ 108,    0,    0]])

<h4>4. Model with LabelEncoded Data + HyperParameters + GridSearchCV + CrossValidation 

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of ur above random search 
param_grid_2 = {
    'bootstrap': [False],
    'max_depth': [8, 10, 12],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 50, 75]
}

# Instantiate the grid search model
model_W_Le_Hp_Grd = GridSearchCV(estimator = model_W_Le_Hp, param_grid = param_grid_2, 
                          cv = 3, n_jobs = -1, verbose = 2)
model_W_Le_Hp_Grd.fit(x1_train, y1_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 15.3min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 25.4min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [ ]:
dump(model_W_Le_Hp_Grd, '/content/models/RF_model_W_Le_Hp_Grd.joblib') 
#model_W_Le_Hp_Grd=load('/content/models/RF_model_W_Le_Hp_Grd.joblib')

['/content/models/RF_model_W_Le_Hp_Grd.joblib']

In [ ]:
#View the best parameters of the best estimator
model_W_Le_Hp_Grd.best_params_

{'bootstrap': False,
 'max_depth': 12,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 12,
 'n_estimators': 50}

In [ ]:
#Test ur model with the best estimator 
best_Mdl_W_Le_Hp_Grd=model_W_Le_Hp_Grd.best_estimator_
best_Mdl_W_Le_Hp_Grd.score(x1_test, y1_test)

0.8713

In [ ]:
metrics.confusion_matrix(y1_test,best_Mdl_W_Le_Hp_Grd.predict(x1_test),labels=[3,2,1])

array([[8710,    3,    0],
       [1176,    3,    0],
       [ 107,    1,    0]])

<h3>Accuracy Description After Adding More Features + Performing Sampling:-
<br>-------------------------------------------------------------------------------------------------
<h4>1. Model with OneHotEncoded Data -------------------- 86.75%
<br>2. Model with LabelEncoded Data --------------------- 86.8%
<br>3. Model with LabelEncoded Data + HyperParameters + RandomizedSearchCV + CrossValidation --------- 87.13%
<br>4. Model with LabelEncoded Data + HyperParameters + GridSearchCV + CrossValidation --------------- 87.13%
<br>
<br>From the Confusion matrices calucalated in each of the above cases you can now verify that our Trained classifier is not providing well suited results for Accident_Severity type 2 and 1. Well it might be for 2 below reasons:
<br>
<br>    1. The Training Dataset is not Balanced
<br>
<br>    2. We might not have enough features which can comfortably classify the results.</h4>

<h4>Lets try to fix the above 2 issue points first and then apply our RandomForestClassifier. 
<br>Please refer to RandomForest_Implementation_2.0.ipynb for further exploration.</h4>
    